In [56]:
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import pyvista

import basix

import ufl

from dolfinx import mesh, fem, plot

In [57]:
domain = mesh.create_unit_square(MPI.COMM_WORLD, 16,16)
element = basix.ufl.element("Lagrange", domain.basix_cell(), 1)
V = fem.FunctionSpace(domain, element)

In [59]:
uD = fem.Function(V)
uD.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)
facets = mesh.locate_entities_boundary(domain, 1, lambda x: np.full(x.shape[1], 0))
boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = fem.dirichletbc(uD, boundary_dofs)

In [60]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [61]:
f = fem.Constant(domain, PETSc.ScalarType(-6))

In [62]:
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

In [63]:
from dolfinx.fem.petsc import LinearProblem
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [64]:
pyvista.start_xvfb()
pyvista.set_jupyter_backend('client')
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()

Widget(value='<iframe src="http://localhost:46335/index.html?ui=P_0x7f1b75f13ca0_12&reconnect=auto" class="pyv…

In [65]:
warped = u_grid.warp_by_scalar()
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)
if not pyvista.OFF_SCREEN:
    plotter2.show()


Widget(value='<iframe src="http://localhost:46335/index.html?ui=P_0x7f1b75b838b0_13&reconnect=auto" class="pyv…